In [ ]:
import numpy as np
import torch
import random
import json

In [ ]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
import os
import ipywidgets as widgets


# latex text in plots
usetex = True
if usetex:
    rc('text', usetex=True)
else:
    rc('text', usetex=False)

#change plt parameter globally 
plt.rcParams.update({'font.size': 18,
                     'xtick.labelsize' : 16,
                     'ytick.labelsize' : 16})

In [ ]:
parent = "_trainings"
children = sorted(os.listdir(parent), reverse=True)

In [ ]:
widget = widgets.Dropdown(
    options=children,
    description='Directory:',
    disabled=False,
)
widget

In [ ]:
path = os.path.join(parent, widget.value)
print("Load results from ", path)

In [ ]:
with open(os.path.join(path, "net_config.json")) as dict_file:
    network_config = json.load(dict_file)
print(network_config)

In [ ]:
loss = np.load(os.path.join(path, "loss.npy"))
dce = np.load(os.path.join(path, "dce.npy"))
loss_val = np.load(os.path.join(path, "loss_val.npy"))
dce_val = np.load(os.path.join(path, "dce_val.npy"))

In [ ]:
print(np.mean(dce_val[80:]))

In [ ]:
n_ep = len(loss)
epochs = np.arange(1,n_ep+1)
print("Total number of epochs: ", n_ep)
plt.figure()
plt.plot(epochs, loss, label="Training")
plt.plot(epochs, loss_val, label="Validation")
#plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Mean Loss")
plt.show()
plt.figure()
plt.plot(epochs, dce, label="Training")
plt.plot(epochs, dce_val, label="Validation")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Mean Dice Score")
plt.show()

In [ ]:
save_image = False

fig, ax1 = plt.subplots()

color = "C1" #'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
ax1.plot(epochs, loss, "--", color=color)
ax1.plot(epochs, loss_val, color=color)
#ax1.tick_params(axis='y', labelcolor=color)
#ax1.set_ylim(0.74,0.77)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = "C0" #'tab:blue'
ax2.set_ylabel('Dice Score', color=color)  # we already handled the x-label with ax1
ax2.plot(epochs, dce, "--", color=color)
ax2.plot(epochs, dce_val, color=color)
#ax2.tick_params(axis='y', labelcolor=color)
#ax2.set_ylim(0.23,0.27)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
if save_image:
    plt.savefig(f"images/{widget.value}.pdf", bbox_inches='tight')
plt.show()

# Show Predictions

In [ ]:
from datasets import LoadDataset, CustomOutput
from datasets.custom_output import image_tensor, float_mask, bounding_boxes
from torch.utils.data import DataLoader, Subset
from trafo.randomize.default_augmentation import default_augmentation_only_geometric

loaded_data = LoadDataset("_data/preprocessed256_new", image_dtype=float,
                          label_dtype=float)

dataset_plain = CustomOutput(loaded_data, image_tensor, float_mask)
dataset_aug = CustomOutput(loaded_data, image_tensor, bounding_boxes, trafo=default_augmentation_only_geometric)

In [ ]:
"""# get good split of dataset -> dividable by batch_size
batch_size = 16
l = len(dataset)
x = l // (batch_size *6 )
split = [x * batch_size * 5, l - x * batch_size * 5]
print(split)
train_set, val_set = torch.utils.data.random_split(dataset, split)""";

from sklearn.model_selection import train_test_split

batch_size = 16

# get good split of dataset -> dividable by batch_size
l = len(dataset_aug)
indices = list(range(l))
train_size = (l // (batch_size * 6)) * batch_size * 5
val_size = l - train_size
print("Training: ", train_size, "Validation: ", val_size)
train_indices, val_indices = train_test_split(indices, random_state=4, train_size=train_size, test_size=val_size)

train_set = Subset(dataset_aug, train_indices)
val_set = Subset(dataset_plain, val_indices)

In [ ]:
dataloader_train = DataLoader(train_set, batch_size=batch_size,
                        shuffle=True, num_workers=0)#, pin_memory = True)
dataloader_val = DataLoader(val_set, batch_size=batch_size,
                        shuffle=True, num_workers=0, pin_memory = True)

In [ ]:
from network.unet import Unet
from network.Model import OurModel
import torch.nn as nn

In [ ]:
criterion = nn.BCELoss() # use binary cross entropy loss!
network = Unet(batch_norm=True)

In [ ]:
Model = OurModel(name = "unet", network= network, criterion= criterion, path_dir = path, lr = 0.001,
                 verbose = True, segmentation=True, use_cuda=False)
Model.load_weights(f"/unet_e100.ckpt")

In [ ]:
for x,y in dataloader_val:
    Model.network.eval()
    y_hat = Model.network(x.float())
    break

y_hat = y_hat.cpu()
y_hat = y_hat.detach().numpy()
y = y.numpy()
fig, ax = plt.subplots(len(y_hat),3, figsize=(10, 100))
for i in range(len(y_hat)):
    ax[i,0].imshow(y_hat[i])
    ax[i,1].imshow(np.round(y_hat[i]))
    ax[i,2].imshow(y.squeeze()[i])